# Ship Damage Odds
Should I roll the dice? This notebook provides the odds that will help Seafall players make that decision. We'll be using [combinations](https://en.wikipedia.org/wiki/Combination) to do the heavy lifting of the calculations. Dice rolls can be [framed](http://math.stackexchange.com/questions/900672/how-many-combinations-from-rolling-5-identical-dice) as [stars and bars](http://math.stackexchange.com/questions/208377/combination-with-repetitions) problems.

In [ ]:
%matplotlib inline

import matplotlib
import matplotlib.pyplot
import numpy
import pandas
import scipy.misc
import scipy.special
import seaborn

## A cursed throw during the prologue
I went to explore a region on the first island, my flag ship being supported by the second ship in my fleet. With the Woodsman as my advisor I would have 4 dice to roll where I needed three successes to avoid damage to my ship. I cast my dice and recoiled once they settled on 3 blanks and merely 1 success. I felt this had to have been some rather unlucky outcome. Was I cursed?

In [ ]:
# FWIW, total number of unique outcomes from rolling 4, 6-sided dice.
scipy.misc.comb(6, 4, repetition=True) # scipy.misc.comb(# of faces, # of dice, repitition=True)
# permutation
scipy.special.perm(4,2)
# combination: asks, "Given 4 dice, how many instances are there where 2 dice ended up with the same face value."
scipy.misc.comb(4,2)

In [ ]:
cursed_probability = (1.0/3)**3 * (2.0/3) * scipy.misc.comb(4,1) + (1.0/3)**4
print cursed_probability

In [ ]:
def prob_blank(num_blank, total_dice):
    if total_dice < num_blank:
        p = 0.0
    else:
        p = (1.0/3)**num_blank * \
        (2.0/3)**(total_dice-num_blank) * \
        scipy.misc.comb(total_dice, total_dice - num_blank)
    return p

In [ ]:
cursed_probability = prob_blank(3,4) + prob_blank(4,4)
print cursed_probability

In [ ]:
def prob_blank_or_worse(num_blank, total_dice):
    if total_dice < num_blank:
        p = 0.0
    else:
        rng_blank = numpy.arange(num_blank, total_dice + 1)
        array_blank = [prob_blank(n, total_dice) for n in rng_blank]
        p = numpy.sum(array_blank)
    return p

In [ ]:
cursed_probability = prob_blank_or_worse(1,4)
print cursed_probability

# SDO matrix
There are 15 custom dice within Seafall, so a matrix should be large enough to assess the situation where all 15 are rolled. Since the early game does not use near this many dice, several visualizations will be created.

In [ ]:
import itertools
row_blank = numpy.arange(1,16) # number of dice showing the blank face
col_dice = numpy.arange(1,16) # number of dice in the pool
sdo = numpy.zeros((numpy.size(row_blank),numpy.size(col_dice)))
prob = [prob_blank_or_worse(*i) for i in itertools.product(row_blank, col_dice)]
ind = [tuple(numpy.subtract(i,1)) for i in itertools.product(row_blank, col_dice)]
for idx, val in enumerate(ind):
    sdo[val] = prob[idx]

In [ ]:
# http://stackoverflow.com/questions/41474284/how-to-convert-a-matrix-into-column-array-with-pandas-python
sdo_dataframe = pandas.DataFrame(data = sdo, index = row_blank, columns = col_dice)
sdo_dataframe_stack = sdo_dataframe.stack().reset_index().rename(columns = {"level_0" : "number_blank", "level_1" : "number_dice_pool", 0 : "probability_GTE_to_number_blank"})

In [ ]:
sdo_dataframe_stack["mask"] = sdo_dataframe_stack["probability_GTE_to_number_blank"] == 0
sdo_mask = sdo_dataframe_stack.pivot("number_dice_pool", "number_blank", "mask").values

In [ ]:
sdo_pivot = sdo_dataframe_stack.pivot("number_dice_pool", "number_blank", "probability_GTE_to_number_blank")

In [ ]:
seaborn.set(style = "white")
seaborn.set_context("poster")
matplotlib.pyplot.figure(figsize=(24, 18))
cmap = seaborn.cubehelix_palette(n_colors = 6,
                                start = 1.5,
                                rot = 1.5,
                                gamma = 1.5,
                                hue = 1.0,
                                dark = 0.525,
                                light = 0.96,
                                reverse = False,
                                as_cmap = True)
ax = seaborn.heatmap(sdo_pivot, 
                     annot = True, 
                     cmap = cmap,
                     cbar = False,
                     mask = sdo_mask,
                     fmt = ".1%", 
                     linewidths = 1.5)

ax.set_title("Ship Damage Probabilty:\n'Probability at least X many blanks given Y sized dice pool', or\n'Probability of greater than or equal to the number of blanks for a given dice pool'",
            fontsize=32)
ax.set_xlabel("Number of blanks", fontsize=32)
ax.set_ylabel("Size of dice pool", fontsize=32)


fig = ax.get_figure()
fig.savefig("sdo.pdf")

# A daring raid upon my province
The end of the 2nd game was nigh. I had 10 glory and was primed to secure a 2 glory treasure and victory on my next turn. I had the most glory, but it was a close game. Mike and Joe both had 9 glory. I had accumulated two treasures to this point and Mike eyed my vault enviously; I had a target on my back. Unfortunately, my ships were both at sea, so my home port was undefended. Mike took advantage and attacked! He had 6 dice to roll and needed 5 successes to raid my treasure room. I had foreseen this possibility and despite the threat I still liked my chances (even though I held out hope Mike wouldn't attack at all). Here is how the numbers break down: